In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

## close range vrs distance range relative to the trollinde radar.

In [2]:
track = pd.read_csv('../saved-files/threshold-20/all-cells-first_landfall-threshold-20.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,8,188.472610,361.877737,16,20,8,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.248473e+06,...,69.599108,17.070280,8,0 days 00:00:00,57.716745,13.245867,34.567482,18.903717,False,25
1,1,7,192.015026,362.419857,11,20,61,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252015e+06,...,69.566479,17.072855,8,0 days 00:05:00,55.372297,12.974534,35.577388,14.922312,False,20
2,2,6,195.334382,362.598927,12,20,110,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.255334e+06,...,69.536276,17.067031,8,0 days 00:10:00,53.078408,12.780750,34.063904,7.959446,False,15
3,3,6,200.713718,364.586398,14,20,151,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.260714e+06,...,69.485415,17.099990,8,0 days 00:15:00,50.903743,12.708913,36.584485,15.919167,False,10
4,4,5,204.367833,365.550327,18,20,198,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.264368e+06,...,69.451293,17.112668,8,0 days 00:20:00,49.448644,12.845599,38.092506,31.838018,False,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1601,294,28,287.000000,377.915132,4,20,15371,2020-03-14 04:30:00,2020-03-14 04:30:00,-2.347000e+06,...,68.689839,17.162032,3817,0 days 00:00:00,76.883847,15.196207,39.099655,NaN,True,0
1602,295,23,272.000000,280.472589,4,20,15397,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.332000e+06,...,68.918856,14.840230,3822,0 days 00:00:00,58.509558,15.270663,40.109338,20.895209,False,5
1603,296,24,273.478351,282.449118,10,20,15428,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.333478e+06,...,68.903894,14.885362,3822,0 days 00:05:00,58.175847,15.390894,40.106802,19.900005,True,0
1604,296,11,196.864798,399.000000,2,20,15415,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.256865e+06,...,69.479063,17.970580,3826,0 days 00:00:00,81.614861,15.390894,40.106802,NaN,True,0


### Calculate distance relative to trollinde radar

In [3]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of trollinde radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of trollinde radar
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [4]:
distance[:10]
# np.max(distance)

[57.71674471539788,
 55.37229665848925,
 53.07840754609132,
 50.90374343608325,
 49.448644247770126,
 43.072917722113004,
 52.2250554994604,
 125.15621745818207,
 77.89545941478816,
 70.69826156120918]

In [5]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 505 µs, sys: 0 ns, total: 505 µs
Wall time: 511 µs


In [6]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
0,0,8,188.472610,361.877737,16,20,8,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.248473e+06,...,69.599108,17.070280,8,0 days 00:00:00,57.716745,13.245867,34.567482,18.903717,False,25
1,1,7,192.015026,362.419857,11,20,61,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252015e+06,...,69.566479,17.072855,8,0 days 00:05:00,55.372297,12.974534,35.577388,14.922312,False,20
2,2,6,195.334382,362.598927,12,20,110,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.255334e+06,...,69.536276,17.067031,8,0 days 00:10:00,53.078408,12.780750,34.063904,7.959446,False,15
3,3,6,200.713718,364.586398,14,20,151,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.260714e+06,...,69.485415,17.099990,8,0 days 00:15:00,50.903743,12.708913,36.584485,15.919167,False,10
4,4,5,204.367833,365.550327,18,20,198,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.264368e+06,...,69.451293,17.112668,8,0 days 00:20:00,49.448644,12.845599,38.092506,31.838018,False,5


# close domain

In [7]:
# latitude of radar(reference point)
ref_latitude_radar = 69.2412
# longitude of radar
ref_longitude_radar = 16.0030
# first 50km from the trollinde radar on the top
latitude_top_50km = 69.6940 
# first 50km from the trollinde radar on the bottom
latitude_bottom_50km = 68.78878

latitude = ((track['latitude'] >= latitude_bottom_50km) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_top = ((track['latitude'] >= ref_latitude_radar) & (track['latitude'] <= latitude_top_50km))
# from_radar_to_50km_bottom = ((track['latitude'] <= ref_latitude_radar) & (track['latitude'] >= latitude_bottom_50km))

longitude = (track['longitude'] <= ref_longitude_radar)

mask_lat = (latitude  & longitude)

# mask_lat = (from_radar_to_50km_top | from_radar_to_50km_bottom)



close_domain_20 = track[ mask_lat]
close_domain_20

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
12,4,15,282.000000,319.361934,2,20,208,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.342000e+06,...,68.795277,15.762955,108,0 days 00:00:00,50.552468,12.845599,38.092506,7.960427,True,0
109,42,36,272.000000,254.000000,1,20,2726,2020-03-13 07:30:00,2020-03-13 07:30:00,-2.332000e+06,...,68.937866,14.193857,726,0 days 00:00:00,79.409520,16.072529,40.610380,10.945538,False,55
110,43,39,273.782811,255.000000,2,20,2806,2020-03-13 07:35:00,2020-03-13 07:35:00,-2.333783e+06,...,68.920982,14.215083,726,0 days 00:05:00,79.496564,16.160648,39.605794,8.955254,False,50
111,44,42,275.000000,252.538127,2,20,2891,2020-03-13 07:40:00,2020-03-13 07:40:00,-2.335000e+06,...,68.911550,14.152807,726,0 days 00:10:00,82.193670,16.221123,40.106802,17.910835,False,45
112,45,40,278.487781,254.662019,12,20,2972,2020-03-13 07:45:00,2020-03-13 07:45:00,-2.338488e+06,...,68.878420,14.198397,726,0 days 00:15:00,82.368148,16.042307,40.109338,18.905844,False,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,292,23,244.000000,311.000000,1,20,15301,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.304000e+06,...,69.147614,15.651495,3809,0 days 00:00:00,17.369328,14.985573,38.596077,5.970224,False,5
1593,293,21,246.713605,313.000000,4,20,15333,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.306714e+06,...,69.121197,15.694082,3809,0 days 00:05:00,18.105814,15.036292,38.598690,NaN,True,0
1594,292,28,270.000000,314.287561,2,20,15306,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.330000e+06,...,68.908776,15.668897,3810,0 days 00:00:00,39.316062,14.985573,38.596077,10.944466,True,0
1602,295,23,272.000000,280.472589,4,20,15397,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.332000e+06,...,68.918856,14.840230,3822,0 days 00:00:00,58.509558,15.270663,40.109338,20.895209,False,5


In [8]:
%%time
cells_relative_time_to_radar_close = [];

for index, cell_id in enumerate(np.unique(close_domain_20['cell'])):
    if cell_id:
        # get all the closed domain cell id
        cells = close_domain_20[close_domain_20['cell'] == cell_id]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar = all_times[::-1][0]
        relative_time = np.subtract(time_cell_hit_radar, all_times)
        # convert to seconds 
        relative_time = relative_time.seconds // 60
        # print(cell_id, relative_time)
        cells_relative_time_to_radar_close.append(relative_time)

CPU times: user 122 ms, sys: 0 ns, total: 122 ms
Wall time: 130 ms


In [9]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_close = pd.Series([item for sublist in cells_relative_time_to_radar_close for item in sublist])
close_domain_20['relative-time-to-LF-close'] = cells_relative_times_to_radar_close.values

/glade/scratch/noteng/ipykernel_44328/49986997.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_domain_20['relative-time-to-LF-close'] = cells_relative_times_to_radar_close.values


In [10]:
close_domain_20 = close_domain_20.reset_index(drop=True)

In [11]:
close_domain_20.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time,relative-time-to-LF-close
0,4,15,282.000000,319.361934,2,20,208,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.342000e+06,...,15.762955,108,0 days 00:00:00,50.552468,12.845599,38.092506,7.960427,True,0,0
1,42,36,272.000000,254.000000,1,20,2726,2020-03-13 07:30:00,2020-03-13 07:30:00,-2.332000e+06,...,14.193857,726,0 days 00:00:00,79.409520,16.072529,40.610380,10.945538,False,55,30
2,43,39,273.782811,255.000000,2,20,2806,2020-03-13 07:35:00,2020-03-13 07:35:00,-2.333783e+06,...,14.215083,726,0 days 00:05:00,79.496564,16.160648,39.605794,8.955254,False,50,25
3,44,42,275.000000,252.538127,2,20,2891,2020-03-13 07:40:00,2020-03-13 07:40:00,-2.335000e+06,...,14.152807,726,0 days 00:10:00,82.193670,16.221123,40.106802,17.910835,False,45,20
4,45,40,278.487781,254.662019,12,20,2972,2020-03-13 07:45:00,2020-03-13 07:45:00,-2.338488e+06,...,14.198397,726,0 days 00:15:00,82.368148,16.042307,40.109338,18.905844,False,40,15


In [12]:
len(close_domain_20)

340

# distant domain

In [13]:
ref_latitude_point_top = 69.6940 
ref_latitude_point_bottom = 68.78878
# longitude of radar
ref_longitude_radar1 = 16.0030
latitude_top_50km_to_150km = 70.5978 # 100km from first 50km on the top
latitude_bottom_50km_to_150km = 67.8878 # 100km from first 50km on the bottom

latitude1 = ((track['latitude'] >= latitude_bottom_50km_to_150km) & (track['latitude'] <= latitude_top_50km_to_150km))
longitude1 = (track['longitude'] <= ref_longitude_radar1)

mask_lat = (latitude1 & longitude1)


distant_domain_20 = track[ mask_lat ]
distant_domain_20

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time
6,0,20,283.801945,325.238055,5,20,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.343802e+06,...,68.773477,15.900718,20,0 days 00:00:00,52.225055,13.245867,34.567482,NaN,True,0
7,0,32,343.000000,267.000000,1,20,32,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.403000e+06,...,68.283043,14.378648,32,0 days 00:00:00,125.156217,13.245867,34.567482,7.960506,True,0
8,3,22,307.633381,308.000000,2,20,167,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.367633e+06,...,68.572660,15.428607,92,0 days 00:00:00,77.895459,12.708913,36.584485,6.966574,True,0
12,4,15,282.000000,319.361934,2,20,208,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.342000e+06,...,68.795277,15.762955,108,0 days 00:00:00,50.552468,12.845599,38.092506,7.960427,True,0
26,5,40,362.820767,238.821109,5,20,278,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.422821e+06,...,68.120596,13.679848,131,0 days 00:00:00,156.183980,13.138674,36.584485,9.951196,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1592,292,23,244.000000,311.000000,1,20,15301,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.304000e+06,...,69.147614,15.651495,3809,0 days 00:00:00,17.369328,14.985573,38.596077,5.970224,False,5
1593,293,21,246.713605,313.000000,4,20,15333,2020-03-14 04:25:00,2020-03-14 04:25:00,-2.306714e+06,...,69.121197,15.694082,3809,0 days 00:05:00,18.105814,15.036292,38.598690,NaN,True,0
1594,292,28,270.000000,314.287561,2,20,15306,2020-03-14 04:20:00,2020-03-14 04:20:00,-2.330000e+06,...,68.908776,15.668897,3810,0 days 00:00:00,39.316062,14.985573,38.596077,10.944466,True,0
1602,295,23,272.000000,280.472589,4,20,15397,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.332000e+06,...,68.918856,14.840230,3822,0 days 00:00:00,58.509558,15.270663,40.109338,20.895209,False,5


In [14]:
%%time
cells_relative_time_to_radar_distant = [];

for index, cell_idx in enumerate(np.unique(distant_domain_20['cell'])):
    if cell_idx:
        # get all the closed domain cell id
        cells = distant_domain_20[distant_domain_20['cell'] == cell_idx]
        # convert time to datetime
        all_times = pd.to_datetime(cells['timestr'].values)
        # select the last time
        time_cell_hit_radar1 = all_times[::-1][0]
        relative_time1 = np.subtract(time_cell_hit_radar1, all_times)
        # convert to seconds 
        relative_time1 = relative_time1.seconds // 60
        # print(cell_idx, relative_time1)
        cells_relative_time_to_radar_distant.append(relative_time1)

CPU times: user 241 ms, sys: 0 ns, total: 241 ms
Wall time: 241 ms


In [15]:
# cells_relative_times = pd.Series([item for sublist in cells_relative_time for item in sublist])
cells_relative_times_to_radar_distant = pd.Series([item for sublist in cells_relative_time_to_radar_distant for item in sublist])
distant_domain_20['relative-time-to-LF-distant'] = cells_relative_times_to_radar_distant.values

/glade/scratch/noteng/ipykernel_44328/64511511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distant_domain_20['relative-time-to-LF-distant'] = cells_relative_times_to_radar_distant.values


In [16]:
distant_domain_20 = distant_domain_20.reset_index(drop=True)


In [17]:
distant_domain_20.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,...,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),landfall,relative_time,relative-time-to-LF-distant
0,0,20,283.801945,325.238055,5,20,20,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.343802e+06,...,15.900718,20,0 days 00:00:00,52.225055,13.245867,34.567482,NaN,True,0,0
1,0,32,343.000000,267.000000,1,20,32,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.403000e+06,...,14.378648,32,0 days 00:00:00,125.156217,13.245867,34.567482,7.960506,True,0,0
2,3,22,307.633381,308.000000,2,20,167,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.367633e+06,...,15.428607,92,0 days 00:00:00,77.895459,12.708913,36.584485,6.966574,True,0,0
3,4,15,282.000000,319.361934,2,20,208,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.342000e+06,...,15.762955,108,0 days 00:00:00,50.552468,12.845599,38.092506,7.960427,True,0,0
4,5,40,362.820767,238.821109,5,20,278,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.422821e+06,...,13.679848,131,0 days 00:00:00,156.183980,13.138674,36.584485,9.951196,True,0,0


In [18]:
len(distant_domain_20)

875

In [19]:
len(close_domain_20)

340

In [20]:
len(track)

1606

In [21]:
len(close_domain_20) + len(distant_domain_20)

1215

In [22]:
distant_domain_20['latitude'].max(), distant_domain_20['latitude'].min(),close_domain_20['latitude'].max(), close_domain_20['latitude'].min()

(69.86908721923828, 67.89249393038152, 69.68209301894343, 68.78943133109364)

In [23]:
distant_domain_20['longitude'].max()

16.001475344760582

In [24]:
close_domain_20.to_csv('../saved-files/threshold-20/close-domain-threshold-20-LF.csv', index=False)
distant_domain_20.to_csv('../saved-files/threshold-20/distant-domain-threshold-20-LF.csv', index=False)

# END